In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error
shap.initjs()
import json
import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from collections import Counter

In [61]:
records = []

with open("medicina.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

# Crea DataFrame
medicina = pd.DataFrame(records)

print(medicina.head())

  hidden_id table context                                           question  \
0    289274                La causa più frequente di angioedema isolato d...   
1    289275                Gli autoanticorpi anti topoisomerasi I sono al...   
2    289276                   Le crioglobuline possono essere costituite da:   
3    289277                La normalità dell'esame emocromocitometrico co...   
4    289278                Nell'ambito dei linfociti T circolanti, normal...   

                                             options image answer     macro  \
0  [{'value': 'A', 'text': 'Deficit di C1-inibito...            D  Medicina   
1  [{'value': 'A', 'text': 'LES', 'image': ''}, {...            D  Medicina   
2  [{'value': 'A', 'text': 'Una immunoglobulina m...            D  Medicina   
3  [{'value': 'A', 'text': 'Deficit dell'immunità...            E  Medicina   
4  [{'value': 'A', 'text': 'Linfociti TCD4+', 'im...            A  Medicina   

                                        genC

In [62]:
medicina["topic"].unique()

array(['Allergologia e immunologia clinica', 'Dietetica', 'Farmacologia',
       'Fisiologia', 'Medicina generale', 'Morfologia', 'Patologia',
       'Prevenzione e deontologia medica'], dtype=object)

In [63]:
medicina["topic"].value_counts()

topic
Medicina generale                     307800
Fisiologia                              9299
Patologia                               7395
Prevenzione e deontologia medica        4180
Farmacologia                            3075
Morfologia                              2710
Allergologia e immunologia clinica      1050
Dietetica                                525
Name: count, dtype: int64

In [64]:
medicina["title"].unique()

array(['Esercitati con i quiz di "Allergologia e immunologia clinica" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2012-13"',
       'Esercitati con i quiz di "Allergologia e immunologia clinica" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2010-11"',
       'Esercitati con i quiz di "Allergologia e immunologia clinica" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2011-12"',
       'Esercitati con i quiz di "Dietetica" del concorso "22 procedure Comune di Roma - 57 Dietisti"',
       'Esercitati con i quiz di "Farmacologia" del concorso "MIUR: Esame di Stato professione di Medico 2019/20"',
       'Esercitati con i quiz di "Farmacologia" del concorso "MIUR - Esame di Stato professione di Medico 2018/19"',
       'Esercitati con i quiz di "Farmacologia" del concorso "MIUR - Esame di Stato professione di Medico 2015/16 I sessione"',
       'Esercitati con i quiz di "Farmacologia" del concorso "MIUR - Esame di Stato profession

In [65]:
medicina["title"].value_counts()

title
Esercitati con i quiz di "Medicina generale" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2012-13"                            297000
Esercitati con i quiz di "Medicina generale" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2011-12"                              5400
Esercitati con i quiz di "Medicina generale" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2010-11"                              5400
Esercitati con i quiz di "Fisiologia" del concorso "MIUR - Esame di Stato professione di Medico 2014/15 I e II sessione"                             1170
Esercitati con i quiz di "Fisiologia" del concorso "MIUR - Esame di Stato professione di Medico 2015/16 I sessione"                                  1169
Esercitati con i quiz di "Fisiologia" del concorso "MIUR - Esame di Stato professione di Medico 2013/14 II sessione"                                 1168
Esercitati con i quiz di "Fisiologia" del concorso "MIUR - Esame di St

In [66]:
medicina["url"].value_counts()

url
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R176&id_concorso=176&materiaCode=medicina_generale                     297000
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R113&id_concorso=113&materiaCode=medicina_generale                       5400
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R107&id_concorso=107&materiaCode=medicina_generale                       5400
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R231&id_concorso=231&materiaCode=fisiologia                              1170
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R285&id_concorso=285&materiaCode=fisiologia                              1169
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R193&id_concorso=193&materiaCode=fisiologia                              1168
https://www.quiz-concorsi-online.com/quiz_per_materia.php?pgCode=G28I230R175&id_concorso=175&mat

In [67]:
# Filtra le righe che contengono la frase (case-insensitive)
filtro = medicina["question"].str.contains("Il linfonodo sentinella", case=False, na=False)

# Mostra le righe corrispondenti
medicina[filtro]


,hidden_id,table,context,question,options,image,answer,macro,genCode,text,url,topic,title
325275,2665243,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
326175,2112924,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
327088,898903,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
328035,672720,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
328980,475164,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
329925,516998,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
330849,128089,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."
331770,307279,,,Quale tra le seguenti affermazioni riguardanti...,"[{'value': 'A', 'text': 'la tecnica viene eseg...",,B,Medicina,Medicina__patologia,Patologia,https://www.quiz-concorsi-online.com/quiz_per_...,Patologia,"Esercitati con i quiz di ""Patologia"" del conco..."


In [68]:
filtro = medicina['title'] == 'Esercitati con i quiz di "Medicina generale" del concorso "SSM - Scuole di Specializzazione in Medicina - anno 2012-13"'
domande= medicina[filtro]["question"]
print(domande)

13949     Quali lipoproteine si possono isolare nel plas...
13950     I pazienti con valvola aortica bicuspide si pr...
13951     Quale tra i seguenti è il test più adatto per ...
13952     In quale caso può essere utile l'uso del lipid...
13953     Gli adulti affetti da OSAS (Obstructive Sleep ...
                                ...                        
310944    Il dermotifo e' dovuto ad uno dei seguenti age...
310945    La dispnea che si rileva in corso di acidosi m...
310946                   Nella anemia perniciosa si rileva:
310947    Quale brucella e' sensibile alla tionina (1:800)?
310948    Tutti i cromosomi che non sono cromosomi del s...
Name: question, Length: 297000, dtype: object


In [69]:
#proviamo a mettere domande di altri argomenti (chirurgia maxillo facciale per lo stesso quiz, ma non risulta che compaiano nella lista)
fil = domande.str.contains("Quale di questi criteri clinici non rientra nella definizione di ischemia critica degli arti inferiori:", case=False, na=False)
domande[fil]


Series([], Name: question, dtype: object)

In [71]:
#faccio un dataset con solo le domande che non del 2012-2013,
#  filtro per vedere quante ce nel sono delle 297000.
#  prima verifico che non ci siano doppioni
domande_uniche = domande.drop_duplicates()
print(len(domande_uniche))

print(domande_uniche)


5379
13949    Quali lipoproteine si possono isolare nel plas...
13950    I pazienti con valvola aortica bicuspide si pr...
13951    Quale tra i seguenti è il test più adatto per ...
13952    In quale caso può essere utile l'uso del lipid...
13953    Gli adulti affetti da OSAS (Obstructive Sleep ...
                               ...                        
19344    Il dermotifo e' dovuto ad uno dei seguenti age...
19345    La dispnea che si rileva in corso di acidosi m...
19346                   Nella anemia perniciosa si rileva:
19347    Quale brucella e' sensibile alla tionina (1:800)?
19348    Tutti i cromosomi che non sono cromosomi del s...
Name: question, Length: 5379, dtype: object


In [72]:


# Supponiamo che la tua lista si chiami domande_lista
domande_lista = domande.tolist()

# Conta le occorrenze di ogni domanda
conteggi = Counter(domande_lista)

# Filtra solo le domande duplicate (cioè quelle con più di 1 occorrenza)
duplicati = {domanda: count for domanda, count in conteggi.items() if count > 1}

# Stampa i duplicati e quante volte compaiono
for domanda, count in duplicati.items():
    print(f'"{domanda}" è duplicata {count} volte')




"Quali lipoproteine si possono isolare nel plasma prelevato da un soggetto normale dopo 12 ore di digiuno?" è duplicata 55 volte
"I pazienti con valvola aortica bicuspide si presentano con uno o piu' dei seguenti problemi tranne:" è duplicata 55 volte
"Quale tra i seguenti è il test più adatto per la valutazione funzionale globale nel paziente con broncopneumopatia cronica ostruttiva?" è duplicata 55 volte
"In quale caso può essere utile l'uso del lipidogramma?" è duplicata 55 volte
"Gli adulti affetti da OSAS (Obstructive Sleep Apnea Syndrome) sono:" è duplicata 55 volte
"Tutte le seguenti situazioni sono compatibili con diagnosi di stenosi aortica tranne:" è duplicata 55 volte
"Cosa differenzia l'anoressia nervosa dalla bulimia nervosa:" è duplicata 55 volte
"Quale dei seguenti tipi di incontinenza urinaria persistente è nettamente più frequente nel sesso femminile?" è duplicata 55 volte
"Non e' tra le attivita' della trombina:" è duplicata 55 volte
"Quale è la frequenza stimata dell